Packages:
- python=3.8.8=hdb3f193_4
- matplotlib=3.5.0=pypi_0
- numpy=1.24.4=pypi_0
- pandas=1.2.0=py38ha9443f7_0
- tick=0.7.0.1=pypi_0
- pytorch=1.13.1=py3.8_cuda11.7_cudnn8.5.0_0
- pytorch-cuda=11.7=h67b0de4_1
- tensorflow 

In [1]:
import torch
import matplotlib.pyplot as plt

In [54]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# simulation

In [3]:
from simulate import Hawkes_Shot_Noise_Simulate as HSNS

In [4]:
import numpy as np

In [5]:
param = {'alpha': np.array([[0.0, 0.15, 0.4 , 0. ],
                            [0.15, 0.0, 0. , 0.4 ],
                            [0.2 , 0.0, 0.0, 0.4 ],
                            [0.0, 0.2 , 0.4 , 0.0]]),
 'beta': 100,
 'endo_baseline': np.array([0.1 , 0.1 , 0.2, 0.2]),
 'exo_baseline': np.array([0.2, 0.2]),
 'delay': 0.01 # Δ ~ np.random.exponential(delay)
}

In [6]:
simu = HSNS(dim_endo=4, # observable dimension
            dim_exo=2, # shot noise
            verbose=False)

In [7]:
simu.set_parameters(**param)

In [8]:
ts, labels = simu.simulate(
            end_time=10000, n_realization=10)

Simulating...:   0%|          | 0/10 [00:00<?, ?it/s]

In [9]:
simu.compute_cumulants()

In [10]:
simu.L

array([0.79069767, 0.79069767, 0.93023256, 0.93023256])

In [11]:
simu.C

tensor([[1.8425, 1.0212, 1.6418, 1.2201],
        [1.0212, 1.8425, 1.2201, 1.6418],
        [1.6418, 1.2201, 2.3760, 1.7977],
        [1.2201, 1.6418, 1.7977, 2.3760]])

In [12]:
simu.K # K[i,i,j]

tensor([[ 6.7879,  3.8581,  7.0633,  5.5565],
        [ 3.8581,  6.7879,  5.5565,  7.0633],
        [ 6.1525,  4.7278, 10.6285,  8.5420],
        [ 4.7278,  6.1525,  8.5420, 10.6285]])

# Estimation

In [13]:
from estimate import Hawkes_Shot_Noise_Estimate

2023-11-03 13:31:06.572604: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-03 13:31:06.862812: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-03 13:31:08.596786: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-03 13:31:08.596950: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

## sparse


In [14]:
from learners import sparse_phi

In [15]:
hsne = sparse_phi(4,2)

In [16]:
hsne.set_data(ts, end_time=10000)

In [17]:
hsne.estimate_cumulants(H=1)

In [18]:
hsne.set_init_values()
hsne.set_variables()

/mnt/nfs/nrdata02-users-data/ruan/ANotebook/HawkesShotNoise/learners.py:277: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.var_phi = torch.tensor(


In [19]:
hsne.fit(max_iter=20000, 
         learning_rate=0.001, 
         tol=1e-8, 
         print_every=1000)

     n_iter  |  objective  |    rel_obj  
-----------------------------------------
          0  |   5.25e+02  |   7.27e-02
       1000  |   8.12e-01  |   9.60e-04
       2000  |   3.04e-01  |   7.66e-04
       3000  |   2.00e-01  |   2.23e-04
       4000  |   1.57e-01  |   2.89e-04
       5000  |   1.11e-01  |    4.1e-04
       6000  |   6.96e-02  |   5.17e-04
       7000  |   4.05e-02  |   5.50e-04
       8000  |   2.35e-02  |   5.52e-04
       9000  |   1.28e-02  |   6.81e-04
      10000  |   5.86e-03  |   8.60e-04
      11000  |   2.79e-03  |   5.48e-04
      12000  |   2.16e-03  |   5.20e-05
      12609  |   2.12e-03  |     0.e+00
      20000  |   2.12e-03  |     0.e+00


In [20]:
# estimated, true
hsne.adjacency, param['alpha']

(array([[0.        , 0.14888087, 0.38820463, 0.        ],
        [0.14888087, 0.        , 0.        , 0.38820463],
        [0.20335104, 0.        , 0.        , 0.39554566],
        [0.        , 0.20335104, 0.39554566, 0.        ]], dtype=float32),
 array([[0.  , 0.15, 0.4 , 0.  ],
        [0.15, 0.  , 0.  , 0.4 ],
        [0.2 , 0.  , 0.  , 0.4 ],
        [0.  , 0.2 , 0.4 , 0.  ]]))

In [21]:
# estimated, true
hsne.endo_baseline, param['endo_baseline']

(array([0.0958378 , 0.09375387, 0.19205034, 0.18383467], dtype=float32),
 array([0.1, 0.1, 0.2, 0.2]))

In [22]:
# loss using true parameters
hsne.objective(var_phi = torch.tensor([0.15,0.4,0.2,0.4]), var_exo_mu=torch.tensor([0.2]))

tensor(0.0954)

In [23]:
# optimal loss 
hsne.objective()

tensor(0.0021, grad_fn=<AddBackward0>)

## full 
### optimize on $\Phi$ and $\mu_X$

In [24]:
from learners import general_phi

In [25]:
hsne1 = general_phi(4,2)

In [26]:
hsne1.set_data(ts, end_time=10000)

In [27]:
hsne1.estimate_cumulants(H=1)

In [28]:
hsne1.set_init_values()
hsne1.set_variables()

In [29]:
hsne1.fit(max_iter=20000, 
         learning_rate=0.001, 
         tol=1e-8, 
         print_every=1000)

     n_iter  |  objective  |    rel_obj  
-----------------------------------------
          0  |   2.54e+00  |   1.47e+00
       1000  |   1.54e-01  |   2.24e-04
       2000  |   1.01e-01  |   7.65e-04
       3000  |   5.29e-02  |   4.77e-04
       4000  |   3.53e-02  |    2.9e-04
       5000  |   2.93e-02  |   1.43e-04
       6000  |   2.49e-02  |   2.13e-04
       7000  |   1.79e-02  |   4.95e-04
       8000  |   8.89e-03  |   4.03e-04
       9000  |   3.37e-03  |   4.29e-03
      10000  |   2.54e-03  |   4.76e-04
      11000  |   1.43e-03  |   6.45e-04
      12000  |   7.77e-04  |   5.48e-04
      13000  |   5.82e-04  |   1.81e-04
      14000  |   5.02e-04  |   1.42e-03
      15000  |   4.47e-04  |   9.26e-05
      16000  |   9.28e-04  |   3.43e-01
      17000  |   3.82e-04  |   2.01e-02
      18000  |   3.48e-04  |    4.5e-05
      19000  |   3.28e-04  |   3.28e-05
      20000  |   3.13e-04  |   4.72e-05


In [30]:
# estimated, true
hsne1.adjacency, param['alpha']

(array([[ 0.02129852,  0.12054231,  0.37720868, -0.02043126],
        [ 0.15858132,  0.00376782,  0.06772324,  0.3577375 ],
        [ 0.23067106, -0.09111248,  0.00304529,  0.4081171 ],
        [ 0.02953047,  0.21797639,  0.4160642 , -0.02104188]],
       dtype=float32),
 array([[0.  , 0.15, 0.4 , 0.  ],
        [0.15, 0.  , 0.  , 0.4 ],
        [0.2 , 0.  , 0.  , 0.4 ],
        [0.  , 0.2 , 0.4 , 0.  ]]))

In [31]:
# estimated, true
hsne1.endo_baseline, param['endo_baseline']

(array([0.10800214, 0.07949491, 0.20506196, 0.18072231], dtype=float32),
 array([0.1, 0.1, 0.2, 0.2]))

In [32]:
# estimated, true
hsne1.exo_baseline, param['exo_baseline']

(array([0.23672195, 0.18290459], dtype=float32), array([0.2, 0.2]))

In [33]:
hsne1.objective(var_phi=torch.tensor(simu.adjacencys[0], dtype=torch.float32), 
               var_exo_mu=torch.tensor(simu.exo_baselines, dtype=torch.float32))

tensor(0.0954)

In [34]:
hsne1.objective()

tensor(0.0003, grad_fn=<AddBackward0>)

### optimize on $R$ and $\mu_X$

In [55]:
from learners import general_R

In [56]:
hsne2 = general_R(4,2)

In [57]:
hsne2.set_data(ts, end_time=10000)

In [58]:
hsne2.estimate_cumulants(H=1)

In [59]:
hsne2.set_init_values()
hsne2.set_variables()

In [60]:
hsne2.fit(max_iter=20000, 
         learning_rate=0.001, 
         tol=1e-8, 
         print_every=1000)

     n_iter  |  objective  |    rel_obj  
-----------------------------------------
          0  |   2.93e+00  |   1.85e+00
       1000  |   1.54e-01  |   2.12e-04
       2000  |   1.08e-01  |   5.71e-04
       3000  |   4.76e-02  |   7.53e-04
       4000  |   3.24e-02  |   1.31e-04
       5000  |   2.79e-02  |   3.87e-04
       6000  |   1.78e-02  |   1.92e-04
       7000  |   1.28e-02  |     6.e-04
       8000  |   7.36e-03  |   4.41e-04
       9000  |   3.96e-03  |   7.74e-04
      10000  |   2.47e-03  |   2.62e-04
      11000  |   1.86e-03  |   3.12e-04
      12000  |   1.31e-03  |   5.77e-04
      13000  |   8.74e-04  |    4.2e-04
      14000  |   6.23e-04  |   2.26e-03
      15000  |   3.94e-04  |   2.17e-04
      16000  |   2.62e-04  |   3.45e-04
      17000  |   1.98e-04  |   2.54e-04
      18000  |   1.94e-04  |   8.26e-05
      19000  |   1.78e-04  |   1.01e-04
      20000  |   2.04e-04  |   4.02e-03


In [61]:
# estimated, true
hsne2.adjacency, param['alpha']

(array([[ 0.06991637, -0.02245208,  0.2559143 ,  0.22864544],
        [ 0.29441583,  0.00519574, -0.14057203,  0.40001604],
        [ 0.23800161,  0.07342392, -0.03173161,  0.38082606],
        [-0.15107147,  0.2223357 ,  0.4478253 ,  0.01417589]],
       dtype=float32),
 array([[0.  , 0.15, 0.4 , 0.  ],
        [0.15, 0.  , 0.  , 0.4 ],
        [0.2 , 0.  , 0.  , 0.4 ],
        [0.  , 0.2 , 0.4 , 0.  ]]))

In [63]:
# estimated, true
hsne2.endo_baseline, param['endo_baseline']

(array([0.07561372, 0.10639544, 0.13916893, 0.23756759], dtype=float32),
 array([0.1, 0.1, 0.2, 0.2]))

In [64]:
# estimated, true
hsne2.exo_baseline, param['exo_baseline']

(array([0.22536628, 0.20290472], dtype=float32), array([0.2, 0.2]))

In [66]:
hsne2.objective(var_R=torch.tensor(simu.R, dtype=torch.float32), 
               var_exo_mu=torch.tensor(simu.exo_baselines, dtype=torch.float32))

tensor(0.0954)

In [67]:
hsne2.objective()

tensor(0.0002, grad_fn=<AddBackward0>)